In [3]:
def smith_waterman(seq1, seq2, match_score=2, mismatch_penalty=-1, gap_penalty=-1):
    """
    Perform local sequence alignment using the Smith-Waterman algorithm.
    
    Parameters:
    - seq1, seq2: Input sequences (strings).
    - match_score: Score for matching characters (default: 2).
    - mismatch_penalty: Penalty for mismatches (default: -1).
    - gap_penalty: Penalty for gaps (default: -1).
    
    Returns:
    - Aligned sequences (seq1_aligned, seq2_aligned) and the alignment score.
    """
    
    # Initialize the scoring matrix and traceback matrix
    m, n = len(seq1), len(seq2)
    score_matrix = [[0] * (n + 1) for _ in range(m + 1)]
    traceback = [[0] * (n + 1) for _ in range(m + 1)]  # 0=stop, 1=diagonal, 2=up, 3=left
    
    max_score = 0          # Track the highest score in the matrix
    max_positions = []     # Store all positions with max_score
    
    # Fill the scoring and traceback matrices
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Calculate match/mismatch score
            if seq1[i - 1] == seq2[j - 1]:
                diagonal_score = score_matrix[i - 1][j - 1] + match_score
            else:
                diagonal_score = score_matrix[i - 1][j - 1] + mismatch_penalty
            
            # Calculate scores for gaps
            up_score = score_matrix[i - 1][j] + gap_penalty
            left_score = score_matrix[i][j - 1] + gap_penalty
            
            # Determine the best score
            score_matrix[i][j] = max(
                0,                # Local alignment: reset if score < 0
                diagonal_score,    # Match/mismatch
                up_score,          # Gap in seq2
                left_score         # Gap in seq1
            )
            
            # Update traceback matrix
            if score_matrix[i][j] == 0:
                traceback[i][j] = 0  # Stop
            elif score_matrix[i][j] == diagonal_score:
                traceback[i][j] = 1  # Diagonal (match/mismatch)
            elif score_matrix[i][j] == up_score:
                traceback[i][j] = 2  # Up (gap in seq2)
            else:
                traceback[i][j] = 3  # Left (gap in seq1)
            
            # Track the maximum score and its positions
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_positions = [(i, j)]
            elif score_matrix[i][j] == max_score:
                max_positions.append((i, j))
    
    # Traceback from the first maximum position (for simplicity)
    i, j = max_positions[0]
    aligned_seq1, aligned_seq2 = [], []
    
    while traceback[i][j] != 0:  # Stop when traceback is 0
        if traceback[i][j] == 1:  # Diagonal (match/mismatch)
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif traceback[i][j] == 2:  # Up (gap in seq2)
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        elif traceback[i][j] == 3:  # Left (gap in seq1)
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
    
    # Reverse the aligned sequences (traceback is backward)
    aligned_seq1 = ''.join(reversed(aligned_seq1))
    aligned_seq2 = ''.join(reversed(aligned_seq2))
    
    return aligned_seq1, aligned_seq2, max_score


# Example usage
if __name__ == "__main__":
    seq1 = "ACCTA"
    seq2 = "AGCTA"
    
    aligned_seq1, aligned_seq2, score = smith_waterman(seq1, seq2)
    
    print(f"Sequence 1: {aligned_seq1}")
    print(f"Sequence 2: {aligned_seq2}")
    print(f"Alignment Score: {score}")

Sequence 1: ACCTA
Sequence 2: AGCTA
Alignment Score: 7
